In [1]:
import sys
sys.path.append('./py/')

In [2]:
import ee
from ParamSelection import Landsat
from ProductSelection import Product
from IndicesAdd import Indices
from AssetsFilter import Assets

In [3]:
service_account = 'glodal-landcover-mapping@rohit-81.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'glodal_gee.json')
ee.Initialize(credentials)

In [4]:
ROI = {'cityName': 'Chittagong', 'LatMax': 22.4525957509930443, 'LatMin': 22.1544461679352587, 'LonMax': 91.9732519334708627, 'LonMin': 91.7009214895644647, 'yearBegin':2020, 'yearEnd':2020, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  1), ee.Filter.lessThanOrEquals('doy',  366)), 'doyFilterLandsat': ee.Filter.dayOfYear(1,366)}

In [5]:
city_name = ROI['cityName']
year_begin = ROI['yearBegin']
year_end = ROI['yearEnd']

In [6]:
def reduce_class_value(feat):
    return feat.set('class', ee.Number(feat.get('class')).subtract(1))

In [7]:
ref_point = ee.FeatureCollection('users/rhtkhati/gt-pt-2019-2021-120m')
ref_point = ref_point.map(reduce_class_value)

In [8]:
kernel = ee.Kernel.gaussian(1)
cloud = 30
spatial_resolution = 120
batch_size = 100
base_distance = 5000
patch_size = 256
buffer_distance = base_distance * (round((spatial_resolution * 0.5 * patch_size)/base_distance) +1)

In [9]:
for year in range(year_begin, year_end+1, 1):
    out_ext = ee.Geometry.Rectangle([ ROI['LonMin'], ROI['LatMin'], ROI['LonMax'], ROI['LatMax']])
    out_ext_center = out_ext.centroid()

    out_ext_buffer = out_ext.buffer(buffer_distance).bounds()

    center_lat = out_ext_center.getInfo()['coordinates'][1]
    center_lon = out_ext_center.getInfo()['coordinates'][0]
    
    landsat_parameters = Landsat(year_begin).select_landsat()
    
    gtYearBegin = year_begin
    gtYearEnd = year_end
    
    landsat_product = Product(landsat_parameters['Landsat'], gtYearBegin, gtYearEnd, ROI['doyFilterLandsat'], out_ext, cloud).select_product()
    
    LABEL_DATA = ref_point.filter(ee.Filter.And(ee.Filter.greaterThanOrEquals('year', gtYearBegin), ee.Filter.lessThanOrEquals('year', gtYearEnd), ROI['doyFilter']))
    LABEL_DATA = LABEL_DATA.filterBounds(landsat_product['region'])
    
    data_size = LABEL_DATA.size().getInfo()
    train_step = (data_size*0.8)//batch_size
    validate_step = (data_size*0.2)//batch_size
    
    Indices(landsat_parameters, landsat_product).add_indices()
    
    gt_image = landsat_product['image']
    gt_image_int = landsat_product['image_int']
    gt_image_texture = gt_image.addBands(gt_image_int.select(landsat_parameters['TextureBand']).entropy(kernel))

    out_image = landsat_product['image'].clip(out_ext_buffer)
    out_image_int = landsat_product['image_int'].clip(out_ext_buffer)
    out_image_texture = out_image.addBands(out_image_int.select(landsat_parameters['TextureBand']).entropy(kernel))

In [ ]:
import folium

In [10]:
mapid = out_image_texture.getMapId({'bands': [landsat_parameters['FCCbands'][0], landsat_parameters['FCCbands'][1], landsat_parameters['FCCbands'][2]], 'min': 0, 'max': 0.3})
map = folium.Map(location=[center_lat, center_lon]) 
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Out Image Texture',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

NameError: name 'folium' is not defined